In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

In [2]:
ranks = {}
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

In [3]:
car = pd.read_csv("../datasets/cars/New_cars_preprocessed.csv")
car.head()

,EPA Fuel Economy Est - City (MPG),Engine,Drivetrain,Passenger Capacity,Passenger Doors,Wheelbase (in),SAE Net Torque @ RPM,Fuel System,SAE Net Horsepower @ RPM,Displacement,...,Turning Diameter - Curb to Curb (ft),Front Wheel Material,Roadside Assistance Miles/km,Manufacturer,Model year,Category,Front tire width,Front tire aspect ratio,Front tire rim size,MSRP
0,0.333333,0.769231,0.50,0.333333,1.0,0.608427,0.299465,0.0,0.336634,0.266667,...,0.416667,0.25,0.333333,0.0,1.0,0.5,0.279070,0.647059,0.863636,40600.0
1,0.333333,0.769231,0.50,0.333333,1.0,0.608427,0.299465,0.0,0.336634,0.266667,...,0.416667,0.25,0.333333,0.0,1.0,0.5,0.279070,0.647059,0.863636,45500.0
2,0.333333,0.769231,0.50,0.333333,1.0,0.608427,0.299465,0.0,0.336634,0.266667,...,0.416667,0.25,0.333333,0.0,1.0,0.5,0.348837,0.529412,0.909091,43600.0
3,0.333333,0.769231,0.50,0.333333,1.0,0.608427,0.299465,0.0,0.336634,0.266667,...,0.416667,0.25,0.333333,0.0,1.0,0.5,0.279070,0.647059,0.863636,37400.0
4,0.318182,0.769231,0.25,0.333333,1.0,0.608427,0.299465,0.0,0.336634,0.266667,...,0.416667,0.25,0.333333,0.0,1.0,0.5,0.279070,0.647059,0.863636,42600.0


In [4]:
# car = car.drop(['Unnamed: 0'], axis=1)
# car

In [5]:
X_car = car.copy()
y_car = X_car.pop("MSRP")
colnames_car = X_car.columns
colnames = X_car.columns

In [6]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32262 entries, 0 to 32261
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   EPA Fuel Economy Est - City (MPG)     32262 non-null  float64
 1   Engine                                32262 non-null  float64
 2   Drivetrain                            32262 non-null  float64
 3   Passenger Capacity                    32262 non-null  float64
 4   Passenger Doors                       32262 non-null  float64
 5   Wheelbase (in)                        32262 non-null  float64
 6   SAE Net Torque @ RPM                  32262 non-null  float64
 7   Fuel System                           32262 non-null  float64
 8   SAE Net Horsepower @ RPM              32262 non-null  float64
 9   Displacement                          32262 non-null  float64
 10  Trans Description Cont.               32262 non-null  float64
 11  Trans Type     

In [7]:
model = DecisionTreeRegressor()
model.fit(X_car, y_car)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
    
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
cart = DecisionTreeRegressor()
cart.fit(X_car,y_car)
ranks["CART"] = ranking(cart.feature_importances_, colnames);
ranks

In [ ]:
decimalrank_df = pd.DataFrame()

# Join preranks in dataframe
for method_name, rank_list in ranks.items():
    df_Temp = pd.DataFrame(data = {method_name:pd.Series(rank_list)})
    decimalrank_df = pd.concat([decimalrank_df,df_Temp ], axis=1)

rank_df = pd.DataFrame()
# Create new integer ranks
for method_name, rank_list in ranks.items():
    rank_df[method_name] = decimalrank_df[method_name].rank(ascending=False)

rank_df['feature_name'] = rank_df.index

In [ ]:
ranksout = rank_df.melt(id_vars = 'feature_name', var_name='method', value_name='feature_rank')
ranksout['dataset'] = 'cars'
ranksout

In [ ]:
ranksout.to_csv('../data/CART/cars_featureranks_CART.csv', sep=';',index=False)